In [2]:
file_="C:/Users/33623/Ultiwatt Dropbox/david mouquet/D - ULTIVISION INDUSTRIES/4-ARCELORMITTAL/2- DEPLOIEMENT AMF/7-Projets 2023/1-UES FLO GALSA1/3-Données Arcelor/BO-Arrets/types d'arrêt.xlsx"
import pandas as pd
label = pd.read_excel(file_,usecols=['Liste Groupe'])
label.dropna(inplace=True)
data  = pd.read_excel(file_,usecols=['commentaire','Groupe']) 

In [3]:
data

,commentaire,Groupe
0,ATMOSPHERE FOUR,Reste
1,ATMOSPHERE FOUR fuite eau sur bande . conduit...,Reste
2,AUTRES ARRETS PLANNIFIES,Arrets plannifiés/programmés
3,AUTRES ARRETS PLANNIFIES ATTENTE PROGRAMME,Arrets plannifiés/programmés
4,AUTRES ARRETS PLANNIFIES Avance bande.,Arrets plannifiés/programmés
...,...,...
88,TRANSPORT et PONTS,Transport et pont
89,TRANSPORT et PONTS Attente alimantation,Transport et pont
90,TRANSPORT et PONTS idem.,Transport et pont
91,TRANSPORT et PONTS pont SA43 avance bande.,Transport et pont


In [4]:
target_unique = label.index.values
target_names = list(label['Liste Groupe'])
dico_map = {g:t for g,t in zip(target_names,target_unique)}

data['target'] = data['Groupe'].map(dico_map)
target = data['target'].values

In [8]:
import spacy
from spacy import displacy
nlp = spacy.load('fr_core_news_sm')

In [9]:
def lemmatize(text):

   doc = nlp(text)

   tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]

   return ' '.join(tokens)

In [10]:
data['processed_text'] = data.commentaire.apply(lambda txt : lemmatize(txt))

In [11]:
data

,commentaire,Groupe,target,processed_text
0,ATMOSPHERE FOUR,Reste,11,atmosphere four
1,ATMOSPHERE FOUR fuite eau sur bande . conduit...,Reste,11,atmosphere four fuite eau bande conduite cassé
2,AUTRES ARRETS PLANNIFIES,Arrets plannifiés/programmés,0,arret PLANNIFIES
3,AUTRES ARRETS PLANNIFIES ATTENTE PROGRAMME,Arrets plannifiés/programmés,0,arret PLANNIFIES attente PROGRAMME
4,AUTRES ARRETS PLANNIFIES Avance bande.,Arrets plannifiés/programmés,0,arret PLANNIFIES avance bande
...,...,...,...,...
88,TRANSPORT et PONTS,Transport et pont,3,transport PONTS
89,TRANSPORT et PONTS Attente alimantation,Transport et pont,3,transport pont attente alimantation
90,TRANSPORT et PONTS idem.,Transport et pont,3,transport pont idem
91,TRANSPORT et PONTS pont SA43 avance bande.,Transport et pont,3,transport pont pont sa43 avance band


In [12]:
# import and instantiate the vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

# vectorize the lemmatized text
X = cv.fit_transform(data.processed_text)
print(X)

  (0, 5)	1
  (0, 62)	1
  (1, 5)	1
  (1, 62)	1
  (1, 64)	1
  (1, 49)	1
  (1, 13)	1
  (1, 34)	1
  (1, 25)	1
  (2, 4)	1
  (2, 106)	1
  (3, 4)	1
  (3, 106)	1
  (3, 6)	1
  (3, 115)	1
  (4, 13)	1
  (4, 4)	1
  (4, 106)	1
  (4, 7)	1
  (5, 4)	1
  (5, 106)	1
  (5, 58)	1
  (6, 4)	1
  (6, 106)	1
  (6, 7)	1
  :	:
  (87, 62)	1
  (87, 153)	1
  (87, 126)	1
  (87, 163)	1
  (87, 37)	1
  (87, 152)	1
  (88, 154)	1
  (88, 110)	1
  (89, 6)	1
  (89, 109)	1
  (89, 154)	1
  (89, 2)	1
  (90, 70)	1
  (90, 109)	1
  (90, 154)	1
  (91, 7)	1
  (91, 12)	1
  (91, 109)	2
  (91, 154)	1
  (91, 138)	1
  (92, 51)	1
  (92, 109)	1
  (92, 154)	1
  (92, 94)	1
  (92, 136)	1


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X, target)

# 3. Make predictions
yhat = clf.predict(X)

# 4. score
print("Accuracy: ",accuracy_score(target, yhat))

Accuracy:  0.978494623655914


In [27]:
new_text = 'PLANNIFIES'

df_new = pd.DataFrame({'commentaire':['ARRETS PLANNIF','TRANSPORT']})

df_new['processed_text'] =  df_new.commentaire.apply(lambda txt : lemmatize(txt))
X_new = cv.transform(df_new.processed_text)
#class_pred = clf.fit(new_text)
y_new = clf.predict(X_new)

In [28]:
y_new

array([11,  3], dtype=int64)

In [20]:
df_new

,commentaire,processed_text
0,PLANNIFIES,plannifie
1,TRANSPORT,transport


In [19]:
dico_map

{'Arrets plannifiés/programmés': 0,
 'Manque métal amont': 1,
 'Qualité': 2,
 'Transport et pont': 3,
 'Changement matériel immergé': 4,
 'BS': 5,
 'Skin-pass': 6,
 'Soudeuse': 7,
 'Enroulement': 8,
 'Rupture bande': 9,
 'Non renseigné': 10,
 'Reste': 11}

In [18]:
y_new

array([11,  3], dtype=int64)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(comment)
X_train_counts.shape

(93, 180)

In [58]:
count_vect.get_feature_names_out()

array(['13h30', '2m', 'accu', 'alimantation', 'amont', 'apres', 'arret',
       'arrets', 'atmosphere', 'attente', 'au', 'autres', 'avance',
       'avancée', 'axe', 'baffle', 'bande', 'bloqués', 'bobine',
       'bobines', 'bourrage', 'bs1', 'bs2', 'bs3', 'bs4', 'bs5', 'buses',
       'capteur', 'cassé', 'cdr', 'centreur', 'changement', 'chariot',
       'chutages', 'chute', 'chutes', 'ci', 'cisaille', 'cm', 'conduite',
       'consigne', 'contact', 'convertisseur', 'coup', 'coupe', 'creuset',
       'danseur', 'de', 'decaler', 'defl', 'deport', 'deroulage', 'des',
       'difficulté', 'du', 'déclenchement', 'déport', 'eau', 'ejecteurs',
       'electricite', 'embarqué', 'en', 'encocheuse', 'energie',
       'enroulement', 'entr', 'entre', 'entree', 'entretien', 'et',
       'evacuation', 'fait', 'figé', 'fluide', 'four', 'frottement',
       'fuite', 'gestion', 'grains', 'hmi', 'hs', 'huile', 'idem', 'ifg',
       'immerge', 'immergees', 'inducteur', 'intouch', 'jet', 'liaison',
    

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(93, 180)

In [59]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, target)

In [60]:
import numpy as np

predicted = clf.predict(X_train_counts)
np.mean(predicted == target)

0.989247311827957

In [61]:
data['target_predite'] = predicted

In [54]:
data

,commentaire,Groupe,target,target_predite
0,ATMOSPHERE FOUR,Reste,11,11
1,ATMOSPHERE FOUR fuite eau sur bande . conduit...,Reste,11,11
2,AUTRES ARRETS PLANNIFIES,Arrets plannifiés/programmés,0,0
3,AUTRES ARRETS PLANNIFIES ATTENTE PROGRAMME,Arrets plannifiés/programmés,0,0
4,AUTRES ARRETS PLANNIFIES Avance bande.,Arrets plannifiés/programmés,0,0
...,...,...,...,...
88,TRANSPORT et PONTS,Transport et pont,3,3
89,TRANSPORT et PONTS Attente alimantation,Transport et pont,3,3
90,TRANSPORT et PONTS idem.,Transport et pont,3,3
91,TRANSPORT et PONTS pont SA43 avance bande.,Transport et pont,3,3


In [64]:
X_new_counts

<2x180 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [63]:
docs_new = ['AUTRES ARRETS PLANNIFIES', 'enroulem']
X_new_counts = count_vect.transform(docs_new)
#X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted_new = clf.predict(X_new_counts)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, target_names[category]))

'AUTRES ARRETS PLANNIFIES' => Reste
'enroulem' => Reste


In [56]:
target_names

['Arrets plannifiés/programmés',
 'Manque métal amont',
 'Qualité',
 'Transport et pont',
 'Changement matériel immergé',
 'BS',
 'Skin-pass',
 'Soudeuse',
 'Enroulement',
 'Rupture bande',
 'Non renseigné',
 'Reste']